#  A python implementation of the Black Litterman model on the S&P 500 index

###### This work is based on professor Calm O' Cinneide's lecture notes in the  Multi Asset Portfolio Management course at the Columbia MAFN program

###            
###          


###  There are 2 aspects one needs to understand about the Black Litterman model.First one is the statistical structure of the model, and the other is how this statistical proceedure is implemented in formulating the optimal weights of assets in which to invest. Below follows a (very) brief look into the mathematics of the model.

#### Notation: 
#### i) We denote the prior by $f(θ)$. The statistical model (sometimes reffered to as the "likelihood" in certain bayesian inference texts)  by $ f(x|θ)$, and the posterior by $f(θ|x)$
#### The predictive distribution of future returns is denoted by $f(y|x)$
#### ii) We can derive the posterior by $ f(θ|x)= \frac{f(x|θ)f(θ)}{\int f(x|θ)f(θ)dθ} $
#### iii) It is assumed that the prior distribution and the likelihood, are normally distributed with the following means and variances:
### $f(θ)=φ(θ;m,s^2) , f(x|θ)=φ(x;θ,σ_ο^2)$
#### Then it can be shown through straightforward calculations that the posterior also follows a normal distribution:
### $ f(θ|x)= φ(θ; \tilde{m}, \tilde{s}^2)$ where $\tilde{m}=\frac{\frac{m}{s^2}+\frac{x}{σ_ο^2}}{\frac{1}{s^2}+\frac{1}{σ_ο^2}}$ and $\tilde{s}^2= \frac{1}{\frac{1}{s^2}+\frac{1}{σ_ο^2}} $
#### It should be pointed out that the precisions are additive, and that the posterior mean is a precision weighted average of the prior and statistical model means 
#### iv) The predictive distribution $f(y|x)$ can be derived as follows:
####  It is known  that $f(x,y|θ)= f(y|x,θ)f(x|θ)$
#### In order to compute the joint density of $x$ and $y$ we notice that $f(x,y)= \int f(x,y|θ)f(θ)dθ$
##### (we condition on another variable,and later intergrate over all possible values of that variable)
#### combining the results above we get:
### $f(y|x)=\frac{f(x,y)}{f(x)} \implies f(y|x)= \frac{\int f(x,y|θ)f(θ)dθ}{f(x)} \implies f(y|x)=\frac{\int f(y|x,θ)f(x|θ)f(θ)dθ}{f(x)} \implies         f(y|x)= \int f(y|x,θ)f(θ|x)dθ$
####  $f(y|x)$ is used for the predictive distribution instead of $f(y|x,θ)$ which one might think is more intuitive. Reason being we have observed $x$, but not $θ$ . Hence we are averaging over all the possible values of $θ$, with weights of $θ$ being distributed according to the posterior distribution.

#### It can be shown that that $f(y|x)$ is also normally distributed with mean $\tilde{m}$ and variance $ σ_ο^2 + \tilde{s}^2$
#### The predictive distribution has the same mean as the posterior distribution, but an increased variance, to account for the variance of estimation of $θ$, along with the asset's true variance

###  
###  
#### An investor's goal is to maximize the expected value of a utility function who's inputs are a vector of optimal porftolio weights, a vector of excess returns over the benchmark, a covariance matrix and a constant representing risk aversion. One can choose from a variety of utility functions including linear, logarithmic, exponential, however we will use a quadratic as is the standard practice with mean variance preferences. W is used to denote the vector of optimal weights.

###  $ U(w)= w^Tθ - \frac{λw^TΣw}{2} $ 



### Next we focus into how we can incorporate the model into an investment decision making process



In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime

In [2]:
#get a table of all the companies currently in the S&P 500 index

d=pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

#create a list of symbols of all the companies 

symbol=[]
for x in d[0]['Symbol']:
    symbol.append(x)
    
# do some data cleaning,Wikipedia uses dots,whereas Yahoo finance uses dashes (e.g BRK.B vs BRK-B)

for i in range(len(symbol)):
    symbol[i]=symbol[i].replace(".","-")


In [3]:
#for every company retrieve the historical data from 5 years ago up until today

start=datetime.date(2016,8,1)
end=datetime.date.today()

df1=web.DataReader(symbol, 'yahoo',start, end)
df2=pd.DataFrame(df1['Adj Close'])
df=np.log(df2/df2.shift(1))



In [17]:
sigma= df.cov().to_numpy()
        

In [5]:
#calculate the vector of benchmark weights

from pandas_datareader import data

marketcap=data.get_quote_yahoo(symbol)['marketCap']

weights=marketcap.values/marketcap.values.sum()

#### a) The prior distribution represents our belief in the efficiency of the market. We assume that the market is in equilibrium, and so the vector theta of expected excess returns is the one implied by the maximization of the utility function.For an approximation to the market portfolio the S&P 500 index is used. For the covariance matrix $Σ$ we compute the historical returns of the S&P500 over a certain time period, and set the risk aversion constant $λ$ equal to 4. The weights in each stock are computed by dividing the market capitalization of each stock with the total market capitalization of the portfolio.
####
#### b) The likelihood,or statistical model, represents our view.If, for example, our view is that "Apple stock will outperform Tesla by 5%", we create a $p$ vector ([0000......1........00000....-1...000]). Zero weights everywhere, 100% weight in Apple stock, -100% in Tesla stock, and then state that as $pθ=q$

#### c) The posterior mean and variance are denoted by 'thetabl' and 'sigmabl' and are calculated exactly as discussed earlier in the mathematical analysis part. Same thing goes for the variance of the predictive distribution 'sigmapred'. The vector of expected excess returns $θ$ of the predictive distribution,is of course, the posterior or black litterman $θ$

In [77]:
class BlackLitterman:
    def __init__(self,sigma,weights,tau,l,p,q):   
        self.sigma=sigma
        self.weights=weights   #benchmark weights to be imported as a 505x1 vector
        self.tau=tau
        self.l=l
        self.p=p         #the weights of our views
        self.q=q    
        self.thetaimplied= self.sigma@self.weights*self.l
        self.omega=self.tau*self.p.T@self.sigma@self.p
        
        
        
        
        self.sigmabl=np.linalg.inv(np.linalg.inv(self.tau*self.sigma)+self.p@np.linalg.inv(self.omega)@self.p.T)
        self.thetabl= self.sigmabl@(np.linalg.inv(self.tau*self.sigma)@thetaimplied+self.p@np.linalg.inv(self.omega)*self.q)
        self.sigmapred= self.sigma + self.sigmabl
        
        self.predoptimal= 1/l*np.linalg.inv(self.sigmapred)@self.thetabl
        self.bloptimal=1/l*np.linalg.inv(self.sigma)@self.thetabl
        self.naiveoptimal= 1/l*self.q/(self.p.T@self.sigma@self.p)
        
        
            
       

In [45]:
#assume our view is "Apple will outperform Tesla by 5% " 

p=np.zeros((505,1))     #create the vector of weights of our view as explained beforehand
p[symbol.index('AAPL')]=1
p[symbol.index('TSLA')]=-1
q=0.05
tau=1/5  #usually choose tau as 1 over T, where T= amount of years used in computing the historical covariance matrix Σ

In [72]:
model= BlackLitterman(sigma,weights.reshape(505,1),tau,4,p,q)

#### Some things worth noting: 

#### i)Black Litterman optimal weights take are given by $θ$ of the posterior distribution and $Σ$ the historical covariance matrix.Black and Litterman choose to ignore parameter uncertainty. It is somewhat justifiable as the parameter uncertainty tends to be smaller than the inherent underlying risk.This is what how it's generally used by practitioners in the industry as well. While it becomes apparent that the method can no longer be considered purely Bayesian, Black and Litterman themselves never reffered to it as such. What best fits their approach is "Theil's method of mixed estimation". For any reader that might be intrested, further information can be found here: http://sims.princeton.edu/yftp/DummyObs/DumObsPriorSlides.pdf
#### ii) Predictive optimal weights  are computed by the same $θ$ as the posterior, the only difference being that the covariance matrix used, also accounts for parameter uncertainty, adding the estimation error to the asset risk.This is closer to a "pure"  Bayesian approach.
#### iii) The reader might have noticed that "naive optimal" returns a scalar instead a vector which we would expect,since we want to compute the updated weights of the S&P 500 portfolio. It is meant to be multiplied by the weights of our views $p$ forming a long short portfolio,and then added to the benchmark to create the optimal naive portfolio. More information can be found at "The Black Litterman Formula" by Colm O'Cinneide

In [84]:
model.predoptimal

array([[ 2.46858142e-03],
       [ 4.42750748e-03],
       [ 4.31013799e-03],
       [ 2.81663617e-04],
       [ 3.89384462e-03],
       [ 1.52083338e-03],
       [ 4.86355777e-03],
       [ 1.99111365e-03],
       [ 2.80320787e-04],
       [ 3.57740412e-04],
       [ 8.03357520e-04],
       [ 8.48766972e-04],
       [ 1.36709017e-03],
       [ 3.78458955e-04],
       [ 1.75522147e-04],
       [ 3.75348098e-04],
       [ 5.47742573e-04],
       [ 7.89862821e-04],
       [ 9.73533112e-04],
       [ 2.65333332e-04],
       [ 2.97702775e-04],
       [ 8.38023126e-04],
       [ 3.32831678e-02],
       [ 3.32267240e-02],
       [ 1.92694943e-03],
       [ 3.50826214e-02],
       [ 4.10458118e-04],
       [ 4.57565729e-04],
       [ 2.89991150e-04],
       [ 9.26527319e-04],
       [ 2.65141605e-03],
       [ 9.10240316e-04],
       [ 2.28359029e-03],
       [ 5.84914117e-04],
       [ 6.43330766e-04],
       [ 5.06616802e-04],
       [ 6.59620871e-04],
       [ 3.03776571e-03],
       [ 8.4

In [85]:
model.bloptimal

array([[ 2.96229770e-03],
       [ 5.31300898e-03],
       [ 5.17216559e-03],
       [ 3.37996341e-04],
       [ 4.67261354e-03],
       [ 1.82500006e-03],
       [ 5.83626932e-03],
       [ 2.38933638e-03],
       [ 3.36384945e-04],
       [ 4.29288494e-04],
       [ 9.64029024e-04],
       [ 1.01852037e-03],
       [ 1.64050821e-03],
       [ 4.54150745e-04],
       [ 2.10626577e-04],
       [ 4.50417717e-04],
       [ 6.57291087e-04],
       [ 9.47835385e-04],
       [ 1.16823973e-03],
       [ 3.18399998e-04],
       [ 3.57243330e-04],
       [ 1.00562775e-03],
       [ 3.99398014e-02],
       [ 3.98720688e-02],
       [ 2.31233932e-03],
       [ 4.20991457e-02],
       [ 4.92549741e-04],
       [ 5.49078875e-04],
       [ 3.47989379e-04],
       [ 1.11183278e-03],
       [ 3.18169926e-03],
       [ 1.09228838e-03],
       [ 2.74030835e-03],
       [ 7.01896940e-04],
       [ 7.71996919e-04],
       [ 6.07940162e-04],
       [ 7.91545045e-04],
       [ 3.64531885e-03],
       [ 1.0

In [86]:
model.naiveoptimal

array([[10.59706321]])